In [1]:
from pathlib import Path
import czifile
import nd2
import tifffile
import napari
import numpy as np
import os
from skimage import exposure, filters, measure
from scipy.ndimage import binary_erosion
import pyclesperanto_prototype as cle
from stardist.models import StarDist3D
from csbdeep.utils import normalize
import glob
import tensorflow as tf

In [2]:
def read_image (image, slicing_factor):
    """Read raw image microscope files and return a numpy array """
    # Read path storing raw image and extract filename
    file_path = Path(image)
    filename = file_path.stem

    # Extract file extension
    extension = file_path.suffix

    # Read the image file (either .czi or .nd2)
    if extension == ".czi":
        # Stack from .czi (ch, z, x, y)
        img = czifile.imread(image)
        # Remove singleton dimensions
        img = img.squeeze()

    elif extension == ".nd2":
        # Stack from .nd2 (z, ch, x, y)
        img = nd2.imread(image)
        # Transpose to output (ch, z, x, y)
        img = img.transpose(1, 0, 2, 3)

    else:
        print ("Implement new file reader")

    # Apply slicing trick to reduce image size (xy resolution)
    img = img[:, :, ::slicing_factor, ::slicing_factor]

    # Feedback for researcher
    print(f"Image analyzed: {filename}")
    print(f"Original Array shape: {img.shape}")
    print(f"Compressed Array shape: {img.shape}")

    return img, filename

In [3]:
def maximum_intensity_projection (img):

    # Perform MIP on all channels 
    img_mip = np.max(img, axis=1)

    return  img_mip

In [4]:
def extract_nuclei_stack (img, nuclei_channel):

    # Extract nuclei stack from a multichannel z-stack (ch, z, x, y)
    nuclei_img = img[nuclei_channel, :, :, :]

    return nuclei_img

In [5]:
def list_images (directory_path):

    # Create an empty list to store all image filepaths within the dataset directory
    images = []

    # Iterate through the .czi and .nd2 files in the directory
    for file_path in directory_path.glob("*.czi"):
        images.append(str(file_path))
        
    for file_path in directory_path.glob("*.nd2"):
        images.append(str(file_path))

    return images

In [6]:
print("available devices: ",tf.config.list_physical_devices('GPU'))

model = StarDist3D(None, name='3D_seg_stardist_v1.8', basedir='stardist_models')

available devices:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.388346, nms_thresh=0.3.


In [7]:
def segment_nuclei_3d(nuclei_img):
    
    normalized = normalize(nuclei_img)

    nuclei_labels, _ = model.predict_instances(normalized, n_tiles=(6,6,3), show_tile_progress=True)

    return nuclei_labels

In [8]:
# Copy the path where your images are stored, ideally inside the raw_data directory
directory_path = Path("./raw_data/test_data")

# Construct ROI path from directory_path above
roi_path = directory_path / "ROIs"

# Iterate through the .czi and .nd2 files in the raw_data directory
images = list_images(directory_path)

images

['raw_data\\test_data\\HI 1  Contralateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi',
 'raw_data\\test_data\\HI 1  Ipsilateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi',
 'raw_data\\test_data\\MLD_7.5_GFAP_20X 1.500.nd2']

In [9]:
nuclei_channel = 1

In [10]:
img, filename = read_image (images[2], slicing_factor=4)

# Slice the nuclei stack
nuclei_img = extract_nuclei_stack(img, nuclei_channel)

Image analyzed: MLD_7.5_GFAP_20X 1.500
Original Array shape: (2, 13, 1904, 1904)
Compressed Array shape: (2, 13, 1904, 1904)


In [11]:
viewer = napari.Viewer(ndisplay=2)

viewer.add_image(nuclei_img)

Assistant skips harvesting pyclesperanto as it's not installed.


<Image layer 'nuclei_img' at 0x24397f9f8b0>

In [12]:
img.shape

(2, 13, 1904, 1904)

In [13]:
nuclei_labels = segment_nuclei_3d(nuclei_img)

100%|██████████| 18/18 [00:07<00:00,  2.42it/s]


In [ ]:
# Guess tiles logic

#n_tiles = model.predict_instancs(nuclei_img, n_tiles=model._guess_n_tiles(nuclei_img))

In [14]:
viewer.add_image(nuclei_img)
viewer.add_labels(nuclei_labels)

<Labels layer 'nuclei_labels' at 0x2474b086670>

In [15]:
viewer.add_image(img)

<Image layer 'img' at 0x2474b086c10>

In [16]:
# Simulate a cytoplasm by dilating the nuclei and substracting the nuclei mask afterwards
def simulate_cytoplasm(nuclei_labels, dilation_radius = 2, erosion_radius = 0):

    # Dilate nuclei labels to simulate the surrounding cytoplasm
    cyto_nuclei_labels = cle.dilate_labels(nuclei_labels, radius=dilation_radius)
    cyto_nuclei_labels = cle.pull(cyto_nuclei_labels)
    cytoplasm = cyto_nuclei_labels

    # Create a copy of dilated_nuclei to modify
    # cytoplasm = cyto_nuclei_labels.copy()

    # Get unique labels (excluding 0 which is background)
    unique_labels = np.unique(nuclei_labels)
    unique_labels = unique_labels[unique_labels != 0]

    if erosion_radius >= 1:

        # Erode nuclei_labels to maintain a closed cytoplasmic region when labels are touching (if needed)
        eroded_nuclei_labels = cle.erode_labels(nuclei_labels, radius=erosion_radius)
        eroded_nuclei_labels = cle.pull(eroded_nuclei_labels)
        nuclei_labels = eroded_nuclei_labels

    # Iterate over each label and remove the corresponding pixels from dilated_nuclei
    for label in unique_labels:
        # Create a mask for the current label in filtered_nuclei
        mask = (nuclei_labels == label)
        # Set corresponding pixels in resulting_nuclei to zero
        cytoplasm[mask] = 0

    return cytoplasm

cytoplasm_labels = simulate_cytoplasm(nuclei_labels, dilation_radius = 2, erosion_radius = 0)

In [17]:
viewer.add_labels(cytoplasm_labels)

<Labels layer 'cytoplasm_labels' at 0x2475d413610>